In [4]:
import csv

with open('top_movies_list.csv', newline='') as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)

['1', 'Avatar', '2009', '$2881837181']
['2', 'Avengers: Endgame', '2019', '$2797501328']
['3', 'Titanic', '1997', '$2201647264']
['4', 'Star Wars: Episode VII - The Force Awakens', '2015', '$2069521700']
['5', 'Avengers: Infinity War', '2018', '$2048359754']
['6', 'Spider-Man: No Way Home', '2021', '$1915999807']
['7', 'Jurassic World', '2015', '$1671537444']
['8', 'The Lion King', '2019', '$1663250487']
['9', 'The Avengers', '2012', '$1518815515']
['10', 'Furious 7', '2015', '$1515341399']
['11', 'Top Gun: Maverick', '2022', '$1472292161']
['12', 'Frozen II', '2019', '$1450026933']
['13', 'Avengers: Age of Ultron', '2015', '$1402809540']
['14', 'Black Panther', '2018', '$1347597973']
['15', 'Harry Potter and the Deathly Hallows: Part 2', '2011', '$1342359942']
['16', 'Star Wars: Episode VIII - The Last Jedi', '2017', '$1332698830']
['17', 'Jurassic World: Fallen Kingdom', '2018', '$1310466296']
['18', 'Frozen', '2013', '$1281508100']
['19', 'Beauty and the Beast', '2017', '$1273576220